Rugby FL DS

In [1]:
import pandas as pd

In [2]:
# TODO: Import the newest data from the database


In [3]:
results_df = pd.read_csv('results.csv')

results_df['game_date'] = pd.to_datetime(results_df['game_date'])
results_df['season'] = results_df['game_date'].dt.year + 1 * (results_df['game_date'].dt.month >= 9)

# Drop any 0-0 games, has to be OR
drop = results_df[(results_df['home_score'] == 0) & (results_df['away_score'] == 0)]
print(f"Dropping {len(drop)} games with 0-0 scores")
results_df = results_df[(results_df['home_score'] != 0) | (results_df['away_score'] != 0)]

# Drop any game with scores over 200
drop = results_df[(results_df['home_score'] > 200) | (results_df['away_score'] > 200)]
print(f"Dropping {len(drop)} games with scores over 200")
results_df = results_df[(results_df['home_score'] <= 200) & (results_df['away_score'] <= 200)]

# Drop any games with Miami Sharks
drop = results_df[(results_df['home_club'] == 'Miami Sharks')
                  | (results_df['away_club'] == 'Miami Sharks')]
print(f"Dropping {len(drop)} games with Miami Sharks")
results_df = results_df[(results_df['home_club'] != 'Miami Sharks') & (
    results_df['away_club'] != 'Miami Sharks')]

# Fix divisions where missing, set it to the most common for that team
results_df['home_division'] = results_df['home_division'].fillna(
    results_df.groupby('home_club')['home_division'].transform(lambda x: x.value_counts().idxmax()))
results_df['away_division'] = results_df['away_division'].fillna(
    results_df.groupby('away_club')['away_division'].transform(lambda x: x.value_counts().idxmax()))
results_df.dropna(subset=['home_division', 'away_division'], inplace=True)

# Make divisions relative for each team, so if home is at their highest division, it's 0, one lower is 1, etc
results_df['home_divsion'] = results_df['home_division'].astype('category')
results_df['away_divsion'] = results_df['away_division'].astype('category')

results_df['home_club'] = results_df['home_club'].str.strip()
results_df['away_club'] = results_df['away_club'].str.strip()

results_df['home_elo'] = 1500
results_df['away_elo'] = 1500
results_df['elo_change'] = 0
results_df['margin'] = results_df['home_score'] - results_df['away_score']

# Sort by date
results_df = results_df.sort_values(by='game_date')

results_df.to_csv('rugbyfl_data.csv', index=False)

print(f"Final count of games {results_df.shape[0]}")

Dropping 0 games with 0-0 scores
Dropping 1 games with scores over 200
Dropping 17 games with Miami Sharks
Final count of games 3036
